In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import nltk
# nltk.download()

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import PorterStemmer as Stemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 
from nltk import pos_tag

df = pd.read_csv('Tweets.csv')
df

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14635,569587686496825344,positive,0.3487,NaN,0.0000,American,NaN,KristenReenders,NaN,0,@AmericanAir thank you we got on a different f...,NaN,2015-02-22 12:01:01 -0800,NaN,NaN
14636,569587371693355008,negative,1.0000,Customer Service Issue,1.0000,American,NaN,itsropes,NaN,0,@AmericanAir leaving over 20 minutes Late Flig...,NaN,2015-02-22 11:59:46 -0800,Texas,NaN
14637,569587242672398336,neutral,1.0000,NaN,NaN,American,NaN,sanyabun,NaN,0,@AmericanAir Please bring American Airlines to...,NaN,2015-02-22 11:59:15 -0800,"Nigeria,lagos",NaN
14638,569587188687634433,negative,1.0000,Customer Service Issue,0.6659,American,NaN,SraJackson,NaN,0,"@AmericanAir you have my money, you change my ...",NaN,2015-02-22 11:59:02 -0800,New Jersey,Eastern Time (US & Canada)


In [29]:
df_sub = df.loc[:, ['tweet_id', 'airline_sentiment', 'text']]
print(df_sub.dtypes)
df_sub.head(5)

tweet_id              int64
airline_sentiment    object
text                 object
dtype: object


,tweet_id,airline_sentiment,text
0,570306133677760513,neutral,@VirginAmerica What @dhepburn said.
1,570301130888122368,positive,@VirginAmerica plus you've added commercials t...
2,570301083672813571,neutral,@VirginAmerica I didn't today... Must mean I n...
3,570301031407624196,negative,@VirginAmerica it's really aggressive to blast...
4,570300817074462722,negative,@VirginAmerica and it's a really big bad thing...


In [30]:
df_sub['tweet_id'] = df_sub['tweet_id'].astype('str')
print(df_sub.dtypes)
df_sub.head(5)

tweet_id             object
airline_sentiment    object
text                 object
dtype: object


,tweet_id,airline_sentiment,text
0,570306133677760513,neutral,@VirginAmerica What @dhepburn said.
1,570301130888122368,positive,@VirginAmerica plus you've added commercials t...
2,570301083672813571,neutral,@VirginAmerica I didn't today... Must mean I n...
3,570301031407624196,negative,@VirginAmerica it's really aggressive to blast...
4,570300817074462722,negative,@VirginAmerica and it's a really big bad thing...


In [31]:
test_df = df_sub.loc[df_sub['tweet_id'] == '570083898031513600']
test_df

,tweet_id,airline_sentiment,text
778,570083898031513600,negative,@united I'm constantly having challenges with ...


In [32]:
text = str(test_df['text'].values[0])
text

"@united I'm constantly having challenges with upgrades &amp; charges. In order to prevent DVT I have to pay an addtl $180 http://t.co/xC6jQ70r7B"

In [33]:
def text_processing(text):
    processed_text = text.lower() #lowercaseing
    processed_text = re.sub(r'@[A-Za-z0-9]+', '', processed_text) #remove @user mentions
    processed_text = re.sub(r'https?://[A-Za-z0-9./]+', '', processed_text) #remove URLs
    processed_text = re.sub(r'[^\x00-\x7F]+', '', processed_text) #remove non-ASCII characters
    
    stopWords = set(stopwords.words('english'))
    white_list = ["not", "no", "won't", "isn't", "couldn't", "wasn't", "didn't", "shouldn't", 
    "hasn't", "wouldn't", "haven't", "weren't", "hadn't", "shan't", "doesn't",
    "mightn't", "mustn't", "needn't", "don't", "aren't", "won't"]

    words = processed_text.split()
    processed_text = ' '.join([t for t in words if (t not in stopwords.words('english') or t in white_list)]) #remove stop words

    processed_text = ''.join([t for t in processed_text if t not in string.punctuation]) #remove punctuations
    processed_text = ''.join([t for t in processed_text if not t.isdigit()]) #remove numberic numbers
    
    # def stemmer_text(text):
    #     text = word_tokenize(str(text))   # Init the Wordnet Lemmatizer    
    #     st = Stemmer()  
    #     text = [st.stem(t) for t in text]
    #     return (' '.join(text))
    
    # def get_wordnet_pos(treebank_tag):
    #     if treebank_tag.startswith('J'):
    #         return wordnet.ADJ
    #     elif treebank_tag.startswith('V'):
    #         return wordnet.VERB
    #     elif treebank_tag.startswith('N'):
    #         return wordnet.NOUN
    #     elif treebank_tag.startswith('R'):
    #         return wordnet.ADV
    #     else:
    #         return wordnet.NOUN
        
    # use Wordnet(lexical database) to lemmatize text 
    def lemmatize_text(text):
        
        lmtzr = WordNetLemmatizer().lemmatize
        text = word_tokenize(str(text))   # Init the Wordnet Lemmatizer    
        word_pos = pos_tag(text)    
        lemm_words = [lmtzr(sw[0], get_wordnet_pos(sw[1])) for sw in word_pos]
        return (' '.join(lemm_words))

    # processed_text = stemmer_text(processed_text)
    processed_text = lemmatize_text(processed_text)
    return processed_text

print(text_processing(text))

im constantli challeng upgrad amp charg order prevent dvt pay addtl


In [34]:
df_sub['processed_text'] = df_sub['text'].apply(text_processing)
df_sub.head(5)

,tweet_id,airline_sentiment,text,processed_text
0,570306133677760513,neutral,@VirginAmerica What @dhepburn said.,say
1,570301130888122368,positive,@VirginAmerica plus you've added commercials t...,plu ad commerci experi tacki
2,570301083672813571,neutral,@VirginAmerica I didn't today... Must mean I n...,didnt today must mean need take anoth trip
3,570301031407624196,negative,@VirginAmerica it's really aggressive to blast...,realli aggress blast obnoxi entertain guest fa...
4,570300817074462722,negative,@VirginAmerica and it's a really big bad thing...,realli big bad thing


In [35]:
df_sub.loc[:, ('text')].isnull().sum()

0

In [36]:
# df_sub.to_csv('yeyintwin_workshop3.csv', index=False)
# print("Done")

PermissionError: [Errno 13] Permission denied: 'yeyintwin_workshop3.csv'